In [ ]:
# Please note that the parameters below have to be adjusted for specific datasets

from functions import *

In [ ]:
dataset = "MNIST" # either "CIFAR" or "MNIST" or "UCIHAR" or "ISOLET"
max_length = 1000 # Dataset will be shortened to max_length if too large

attempt_using_GPU = True # If GPU is available, it will be used

simple_NN = False # If True, only a very small NN with one hidden layer will be used
                    # Otherwise, a CNN will be used. For CIFAR, a CNN is needed

load_tensor_Hamiltonians = True # All available gates will be loaded as (potentially large) tensors
                                # Depending on your hardware, this becomes problematic at n=8+ qubits

calculate_density_matrices = False # calculate the full 2**n by 2**n density matrices for getting 
                                  # the trace distance (True) or just use 2**n size states (False)

save_data = False # saves loss and gradient to .csv file

load_symbolic_hamiltonians = True # Load available hamiltonians as list of symbolic strings (instead of matrices)

use_symbolic_operations = True # If true, this code will use symbolic operations
                                # see functions ending with _symbolic
if attempt_using_GPU:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
system_check(device, use_symbolic_operations, load_symbolic_hamiltonians, load_tensor_Hamiltonians)

In [ ]:
## Definitions of basic quantum operators, states, and constants
I, X, Y, Z = get_pauli_matrices(device)
zero, one = get_quantum_states(device)
h_bar = 1

In [ ]:
n = 2 # number of qubits
m = n + n + n*(n-1)//2 # number of hamiltonians that will be needed for the system
                       # one X control for every qubit, one Z control for every qubit
                       # 2 pairwise Z controls for every 2 qubits (n choose 2)

shape = (m, 2**n, 2**n) # Shape of the tensor containing all the hamiltonians
T = 1 # Time during which hamiltonians are applied
steps = 5  # Number of discrete values of the activation functions for each hamiltonian
trotter_number = 2 # This is the variable TN is my notes

K = 10 # Number of classes – must be between 2 and the total number of classes given in the dataset

In [ ]:
if load_symbolic_hamiltonians:
    Hamiltonians_symbolic = generate_list_of_hamiltonians_symbolic_form(n)
    
if load_tensor_Hamiltonians:
    Hamiltonians = generate_list_of_hamiltonians_matrix_form(n, shape, I, X, Z, device)

In [ ]:
# See class 'embedder' for details
embedding = embedder(n, Hamiltonians, m, T, steps, trotter_number)

In [ ]:
classes_accepted = [str(x) for x in range(K)]
print("The following classes are added to the dataset:\n", classes_accepted)

In [ ]:
X_train, Y_train, X_test, Y_test, nn_input_dimensions = load_and_preprocess_data(dataset, device, classes_accepted, max_length)

In [ ]:
print("Y_train")
for i in range(max(Y_train)+1):
    if Y_train.count(i) > 0:
        print(i,"occurs",Y_train.count(i),"times")

In [ ]:
print("Y_test")
for i in range(max(Y_test)+1):
    if Y_test.count(i) > 0:
        print(i,"occurs",Y_test.count(i),"times")

In [ ]:
if K != len(set(Y_test)): # Number of distinct classes
    print("There seems to be an error here!")
print("There are", K, "classes in total in the", dataset, "dataset.")

if K != max(Y_test)+1:
    print("Warning! classes in the dataset don't seem to be labeled from 0 to K")

# Defining the Neural Network

In [ ]:
if simple_NN:
    classical_net = Classical_Net_simple(nn_input_dimensions, dataset, K).to(device)
else:
    classical_net = Classical_Net_conv(nn_input_dimensions, dataset, K).to(device)

print("Classical NN:\n\n",classical_net)

In [ ]:
count_parameters(classical_net)

In [ ]:
corrects = 0
predictions = classical_classify(X_train, classical_net)

for i in range(len(Y_train)):
    if predictions[i] == Y_train[i]:
        corrects += 1

print("Training accuracy before classical NN training = ", 100*corrects/len(Y_train),"%")

In [ ]:
corrects = 0
predictions = classical_classify(X_test, classical_net)

for i in range(len(Y_test)):
    if predictions[i] == Y_test[i]:
        corrects += 1

print("Test accuracy before classical NN training = ", 100*corrects/len(Y_test),"%")

In [ ]:
optimizer = optim.SGD(classical_net.parameters(), lr=0.01, weight_decay=1e-1) #MNIST
criterion = nn.CrossEntropyLoss()
loss_history = []
batch_size = len(X_train)
all_labels = torch.zeros((len(Y_train),K))
for i in range(len(Y_train)):
    all_labels[i,Y_train[i]] = 1

In [ ]:
max_epochs = 10000
for epoch in range(max_epochs): # loop over the dataset multiple times
    
    random_indices = random.sample(range(len(X_train)), batch_size)
    
    output = classical_net(X_train[random_indices])
    labels = all_labels[random_indices].type(torch.FloatTensor).to(device)
    
    # zero the parameter gradients
    optimizer.zero_grad()
    
    # forward + backward + optimize
    loss = criterion(output,labels)
    
    loss.backward()
    optimizer.step()

    loss_history.append(loss.detach().to(device='cpu'))
    
print('Finished Training')

In [ ]:
from matplotlib import pyplot as plt
plt.plot(loss_history, label = "Loss function")
plt.ylabel("Training loss")
plt.xlabel("Iteration")
plt.legend()
plt.show()

In [ ]:
corrects = 0
predictions = classical_classify(X_train, classical_net)

for i in range(len(Y_train)):
    if predictions[i] == Y_train[i]:
        corrects += 1

print("Training accuracy after classical NN training = ", 100*corrects/len(Y_train),"%")

In [ ]:
corrects = 0
predictions = classical_classify(X_test, classical_net)

for i in range(len(Y_test)):
    if predictions[i] == Y_test[i]:
        corrects += 1

print("Test accuracy after classical NN training = ", 100*corrects/len(Y_train),"%")

In [ ]:
hidden_layer_size = 120
if simple_NN:
    net = Hybrid_Net_simple(nn_input_dimensions, m, steps, hidden_layer_size, dataset).to(device)
else:
    net = Hybrid_Net_conv(nn_input_dimensions, m, steps, hidden_layer_size, dataset).to(device)
    
print("Quantum NN:\n\n",net)

In [ ]:
nn_output = net(X_train)
nn_output

In [ ]:
activation_functions = torch.reshape(nn_output.type(torch.complex64),(len(nn_output),m,steps))
activation_functions.shape

In [ ]:
test_loss, classes_density_matrices_or_states, _ = nn_loss(X_train, Y_train, net, m, n, steps, K, T, save_data, calculate_density_matrices, use_symbolic_operations, embedding, Hamiltonians, Hamiltonians_symbolic, h_bar, device, trotter_number)
test_loss

In [ ]:
correct_predictions = sum([classify_multiclass(X_train[i], net, classes_density_matrices_or_states, calculate_density_matrices, T, m, n, steps, trotter_number, device, Hamiltonians_symbolic, use_symbolic_operations, embedding, K) == Y_train[i] for i in range(len(Y_train))])
accuracy = (correct_predictions / len(Y_train)) * 100
print("Training accuracy before hybrid NN training = ", accuracy, "%")

In [ ]:
correct_predictions = sum([classify_multiclass(X_test[i], net, classes_density_matrices_or_states, calculate_density_matrices, T, m, n, steps, trotter_number, device, Hamiltonians_symbolic, use_symbolic_operations, embedding, K) == Y_test[i] for i in range(len(Y_test))])
accuracy = (correct_predictions / len(Y_test)) * 100
print("Test accuracy before hybrid NN training = ", accuracy, "%")

In [ ]:
if simple_NN:
    net.fc1.weight = copy.deepcopy(classical_net.fc1.weight)
    net.fc1.bias = copy.deepcopy(classical_net.fc1.bias)
else:
    net.conv1.weight = copy.deepcopy(classical_net.conv1.weight)
    net.conv1.bias = copy.deepcopy(classical_net.conv1.bias)

    net.pool = copy.deepcopy(classical_net.pool)

    net.conv2.weight = copy.deepcopy(classical_net.conv2.weight)
    net.conv2.bias = copy.deepcopy(classical_net.conv2.bias)

    net.fc1.weight = copy.deepcopy(classical_net.fc1.weight)
    net.fc1.bias = copy.deepcopy(classical_net.fc1.bias)

    net.fc2.weight = copy.deepcopy(classical_net.fc2.weight)
    net.fc2.bias = copy.deepcopy(classical_net.fc2.bias)


In [ ]:
test_loss, classes_density_matrices_or_states, _ = nn_loss(X_train, Y_train, net, m, n, steps, K, T, save_data, calculate_density_matrices, use_symbolic_operations, embedding, Hamiltonians, Hamiltonians_symbolic, h_bar, device, trotter_number)
test_loss

In [ ]:
correct_predictions = sum([classify_multiclass(X_train[i], net, classes_density_matrices_or_states, calculate_density_matrices, T, m, n, steps, trotter_number, device, Hamiltonians_symbolic, use_symbolic_operations, embedding, K) == Y_train[i] for i in range(len(Y_train))])
accuracy = (correct_predictions / len(Y_train)) * 100
print("Training accuracy of hybrid NN using classical pre-training = ", accuracy, "%")

In [ ]:
corrects = 0
counter = 0
for i in range(len(Y_train)):
    counter += 1
    if classify_multiclass(X_train[i], net, classes_density_matrices_or_states, calculate_density_matrices, T, m, n, steps, trotter_number, device, Hamiltonians_symbolic, use_symbolic_operations, embedding, K) == Y_train[i]:
        corrects += 1
print("Training accuracy of hybrid NN using classical pre-training = ", 100*corrects/counter,"%")


In [ ]:
corrects = 0
counter = 0
for i in range(len(Y_test)):
    counter += 1
    if classify_multiclass(X_test[i], net, classes_density_matrices_or_states, calculate_density_matrices, T, m, n, steps, trotter_number, device, Hamiltonians_symbolic, use_symbolic_operations, embedding, K) == Y_test[i]:
        corrects += 1
print("Test accuracy of hybrid NN using classical pre-training= ", 100*corrects/counter,"%")

In [ ]:
loss_history = []
grad_history = []

In [ ]:
lr=0.00001
optimizer = optim.SGD(net.parameters(), lr=lr, weight_decay=1e-3)
batch_size = len(X_train)
for epoch in range(10): # loop over the dataset multiple times
    
    # torch.autograd.set_detect_anomaly(True) # Only needed for debugging, not sure if it slows down the code
    
    random_indices = random.sample(range(len(X_train)), batch_size)
    inputs = X_train[random_indices]
    labels = [Y_train[t] for t in random_indices]
    
    # zero the parameter gradients
    start_time = time.time()
    optimizer.zero_grad()
    end_time = time.time()
    print("Time to calculate optimizer.zero_grad() function =", end_time-start_time)
    
    # forward + backward + optimize
    start_time = time.time()
    loss, _ , activation_functions = nn_loss(inputs, labels, net, m, n, steps, K, T, save_data, calculate_density_matrices, use_symbolic_operations, embedding, Hamiltonians, Hamiltonians_symbolic, h_bar, device, trotter_number)
    end_time = time.time()
    print("Time to run nn_loss() function =", end_time-start_time)
    
    start_time = time.time()
    loss.backward()
    end_time = time.time()
    print("Time to calculate loss.backward function =", end_time-start_time)
    
    start_time = time.time()
    optimizer.step()
    end_time = time.time()
    print("Time to calculate optimizer.step()  =", end_time-start_time)

    # Storing the gradient
    if save_data:
        grad_temp_sum = torch.mean(torch.abs(activation_functions.grad)) 
    
    # print statistics
    if save_data:
        print('[%2d] loss: %.9f grad: %.9f' %(epoch, loss, grad_temp_sum))
        grad_history.append(grad_temp_sum)
    else:
        print('[%2d] loss: %.9f' %(epoch, loss))
    
    loss_history.append(loss.item())

print('Finished Training')

In [ ]:
plt.plot(loss_history, label = "Hilbert-Schmidt")
plt.ylabel("Training loss")
plt.xlabel("Iteration")
plt.legend()
plt.show()


In [ ]:
if save_data:
    file_name = "n="+str(n)+" lr="+str(lr)+" loss and grad.csv"
    textfile = open(file_name, "a")
    for i in range(len(loss_history)):
        textfile.write(str(i) + "," + '%.8f,%.8f\n'%(loss_history[i], grad_history[i]))
    textfile.close()

In [ ]:
_, classes_density_matrices_or_states, _ = nn_loss(X_train, Y_train, net, m, n, steps, K, T, save_data, calculate_density_matrices, use_symbolic_operations, embedding, Hamiltonians, Hamiltonians_symbolic, h_bar, device, trotter_number)

In [ ]:
corrects = 0
counter = 0
for i in range(len(Y_train)):
    counter += 1
    if classify_multiclass(X_train[i], net, classes_density_matrices_or_states, calculate_density_matrices, T, m, n, steps, trotter_number, device, Hamiltonians_symbolic, use_symbolic_operations, embedding, K) == Y_train[i]:
        corrects += 1
print("Training accuracy after NN training = ", 100*corrects/counter,"%")


In [ ]:
corrects = 0
counter = 0
for i in range(len(Y_test)):
    counter += 1
    if classify_multiclass(X_test[i], net, classes_density_matrices_or_states, calculate_density_matrices, T, m, n, steps, trotter_number, device, Hamiltonians_symbolic, use_symbolic_operations, embedding, K) == Y_test[i]:
        corrects += 1
print("Test accuracy after NN training = ", 100*corrects/counter,"%")